## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import challenge_g_key

# Configure gmaps API key
gmaps.configure(api_key=challenge_g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.87,113.63,69.80,88,0,23.04,clear sky
1,1,Kavieng,PG,-2.57,150.80,82.35,77,84,10.94,light rain
2,2,Ponta Do Sol,PT,32.67,-17.10,57.20,71,40,17.22,scattered clouds
3,3,Singaraja,ID,-8.11,115.09,80.11,78,100,7.09,overcast clouds
4,4,Anadyr,RU,64.75,177.48,6.80,84,0,8.95,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature you would like?"))
min_temp = float(input("What is the mininum temperature you would like?"))

What is the maximum temperature you would like?75
What is the mininum temperature you would like?60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
vacation_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.87,113.63,69.80,88,0,23.04,clear sky
18,18,Rikitea,PF,-23.12,-134.97,74.57,81,100,10.00,overcast clouds
34,34,Mar Del Plata,AR,-38.00,-57.56,69.01,29,0,17.00,clear sky
35,35,Yanam,IN,16.73,82.22,71.60,94,90,11.41,moderate rain
39,39,Victoria,HK,22.29,114.16,73.99,83,0,1.99,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
vacation_data_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_data_updated_df = vacation_data_df.dropna()
vacation_data_updated_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_data_updated_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,69.80,clear sky,-24.87,113.63,
18,Rikitea,PF,74.57,overcast clouds,-23.12,-134.97,
34,Mar Del Plata,AR,69.01,clear sky,-38.00,-57.56,
35,Yanam,IN,71.60,moderate rain,16.73,82.22,
39,Victoria,HK,73.99,clear sky,22.29,114.16,
41,Front Royal,US,64.00,overcast clouds,38.92,-78.19,
45,San Patricio,US,73.40,broken clouds,28.02,-97.52,
46,Hermanus,ZA,66.99,few clouds,-34.42,19.23,
48,Asyut,EG,68.00,clear sky,27.18,31.18,
51,Airai,TL,63.03,light rain,-8.93,125.41,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": challenge_g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,69.80,clear sky,-24.87,113.63,Hospitality Carnarvon
18,Rikitea,PF,74.57,overcast clouds,-23.12,-134.97,Pension Maro'i
34,Mar Del Plata,AR,69.01,clear sky,-38.00,-57.56,Gran Hotel Mar del Plata habitaciones
35,Yanam,IN,71.60,moderate rain,16.73,82.22,The Regency Hotel
39,Victoria,HK,73.99,clear sky,22.29,114.16,Mini Hotel Central
41,Front Royal,US,64.00,overcast clouds,38.92,-78.19,Hampton Inn Front Royal
45,San Patricio,US,73.40,broken clouds,28.02,-97.52,"Motel 6 Sinton, TX"
46,Hermanus,ZA,66.99,few clouds,-34.42,19.23,Misty Waves Boutique Hotel
48,Asyut,EG,68.00,clear sky,27.18,31.18,Florence Hotel Assiut
51,Airai,TL,63.03,light rain,-8.93,125.41,Juvinal Martin


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
city = hotel_df["City"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))